# Delta-Neutral Simulation 

**TODO: Lots of documentation around what this does, all the parameters, etc.**

**Setup**
1. Navigate to the [Configuration & Scenario Setup](#configuration_setup) section.
1. Update base parameters.
1. Define reward strategy as one of the functions in the [Reward Strategy Definition](#reward_strategy_definition) section.
1. Initialize a trade strategy definition in the [Trade Strategy Definition](#trade_strategy_definition) section.
1. Define a Price Movement Scenario using a function in the [Price Movement Scenario](#price_movement_scenario) section, or create your own.
1. Run
1. View table, text, and plot outputs at the bottom.

**References**: 
* The Best Resource. Used for Benchmarking, Unit Testing, and Key formulas:
[Alpaca Finance Yield Farming Calculator](https://docs.google.com/spreadsheets/d/15pHFfo_Pe66VD59bTP2wsSAgK-DNE_Xic8HEIUY32uQ/edit#gid=650365.25877)
  * 1.1) Pseudo delta-neutral LP farming
  * 3.2) Double-sided Farming
* Impermanent loss equations: [Uniswap Understanding Returns](https://docs.uniswap.org/protocol/V2/concepts/advanced-topics/understanding-returns)
* Details on LPs/DeFi: [DeFi Guide for Newbie (and how to manage risk)](https://www.reddit.com/r/defi/comments/rxj072/defi_guide_for_newbie_and_how_to_manage_risk/)
* [Do fees earned from liquidity pools behave like compound interest?](https://www.reddit.com/r/UniSwap/comments/ldmq18/do_fees_earned_from_liquidity_pools_behave_like/)
  * tl;dr: Every transaction effectively acts as a compounding event. So low-activity pools have lower compounding than high-activity pools. But can be generally treated as compounding. 
* Debt Ratio: [Francium Liquidation Docs](https://docs.francium.io/product/liquidation)
* Overview of different ways to open a Pseudo Delta Neutral position [Revisiting the Fundamentals of Pseudo-Delta Neutral Hedging - DarkRay](https://darkray.medium.com/revisiting-the-fundamentals-of-pseudo-delta-neutral-hedging-4da279caabfa)

**Notes**:
* `token0` always assumed to be the stable, and `token1` is the (risky) asset that is borrowed at a higher rate

**Equations**:

Rough equation to calculate leveraged Yield Farming returns without factoring in compounding, rewards, fees, etc:
```
E = L*P*(1 + r_e/n)^(n*t) - (L-1)*P*(1 + r_b/n)^(n*t)
```
where:
```
E: final equity
L: Leverage amount (1, or >2)
P: Initial principal
r_e: earning interest rate (such that 0.5=50%)
r_b: borrow interest rate (such that 0.5=50%)
n: number of times compounded a year, can assume daily (365.25).
t: number of years
```
This can be used as a comparison to this sim if you set all rates/fees to zero except trading fees, and no variation.

# TODOs
---
- [ ] Spot check rebalance -- check the IL losses. Should have some -- Compare to no-op. Compare to a no-price-change scenario. i.e. is it doing better than if there's no price change at same APRs? If so, can't be possible I think -- See notes in evernote. Something fishy?
- [ ] Fix impermanent loss calculations. Should reset on every rebalance
- [ ] Commit clean
- [ ] Add colab button
- [ ] Set up a downloader on the file (github)
- [ ] Document description

# Pip Installs
---

In [ ]:
!pip install matplotlib plotly pandas pycoingecko

# Imports
---

In [ ]:
import matplotlib.pyplot as plt
import datetime
import json
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
from pycoingecko import CoinGeckoAPI
import requests
from typing import Any, Dict, Optional, Sequence, Tuple
import time

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Functions
---

## Global Defaults

In [ ]:
# Files for fetching APYs using create_history_from_files()
DEFAULT_COINDIX_HISTORY_FILE = "coindix_vault_history_20220601.json"  # For historical pool APYs
DEFAULT_CREAM_HISTORY_FILE = "cream_finance_history_20220601.json"  # For historical borrow APYs

## Support Functions

In [ ]:
def iloss_amt(k: float, r: float) -> Sequence[float]:
    """
    NB: This wasn't in video screenshot, so derived it manually.

    Calculate impermanent loss given constant product formula: x*y = k.
    Ref: https://jamesbachini.com/impermanent-loss/

    Given x*y = k

    x_new = sqrt(k / r)
    y_new = sqrt(k * r)

    Assumes price ratio is price_b / price_a

    Args:
        k: lp constant
        x: price ratio

    Returns:
        iloss_value: Values after impermanent loss calculated, 
            where first element is # of tokens remaining in pool A
            and second element is # tokens in pool B
    """
    iloss_val = []
    print
    x = (k * r) ** 0.5
    y = (k / r) ** 0.5
    iloss_val = [x, y]
    return iloss_val


In [ ]:
def random_walk(n: int, origin: float, bias: float, variance: float, seed: Optional[int] = None) -> np.array:
    """ 
    Generates n-length array from random walk with bias and variance per step. 
    Optionally set seed for deterministic results.
    """
    # Set randon number generator just for this function
    rng = np.random.RandomState(seed)
    r = rng.normal(bias, np.sqrt(variance), n, ) + 1.0
    walk = np.cumprod(r) * origin
    walk = np.insert(walk, 0, origin)[:-1]    
    return walk

In [ ]:
def initialize_price_dataframe(
    n_days: int, token0_price_initial: float, token1_price_initial: float, start_date: str = "2021-01-01"
) -> pd.DataFrame:
    """ 
    Creates simple dataframe with date and two token prices to use as initialization 
    for scenario generators.
    """
    df = pd.DataFrame(
        (
            pd.date_range("2021-01-01", periods=n_days, freq="D"), 
            n_days*[1.0], 
            n_days*[token1_price_initial]
        )
    )
    df = df.T
    df = df.rename({0: "date", 1: "token0_price", 2: "token1_price"}, axis=1)
    df = df.set_index("date")
    return df

In [ ]:
def get_coingecko_price_history(coin_id: str, days: str = "max", interval: str = "daily") -> pd.Series:
    """
    Fetches daily prices for a coingecko ID.
    To get coingecko ID, look for "API id" on the CoinGecko page for that coin
    
    Returns a Seriees indexed on date and price in dollars as the field.
    """
    cg = CoinGeckoAPI()
    p = cg.get_coin_market_chart_by_id(coin_id, vs_currency="usd", days=days, interval=interval)
    df = pd.DataFrame(p["prices"], columns=["date", "price"])
    df["date"] = pd.to_datetime(df["date"]/1000, utc=True, unit="s") 
    df = df.set_index("date", drop=True)
    
    # Remove most recent since it is current moment
    df = df.iloc[:-1, :]
    
    # Convert to a series
    s = df["price"]
    s = s.tz_localize(None)
    
    return s

In [ ]:
def get_binance_klines(
    symbol: str, 
    candle_size: str, 
    start_time: int, 
    end_time: int,
    limit: int
):
    """
    Gets candlesticks from Binance and returns as dataframe.
    
    Args:
        symbol: String representation of pair to fetch
        candle_size: See Binance API. e.g. 1h, 1d ...
        start_time: Start time in epoch seconds
        end_time: end time in epoch seconds
        limit: Limit to number of datapoints to fetch
    """

    BINANCE_KLINES_URL = "https://www.binance.com/api/v3/klines"

    params = dict(
        interval=candle_size,
        symbol=symbol,
        limit=limit
    )
    resp = requests.get(BINANCE_KLINES_URL, params=params)
    resp_json = resp.json()
    df = pd.DataFrame(resp_json)
    df = df.iloc[:, 0:6]
    df = df.rename(
        {
            0: "Time",
            1: "Open",
            2: "High",
            3: "Low", 
            4: "Close",
            5: "Volume",
        }, 
        axis='columns'
    )
    df["Time"] = df["Time"].astype(int) / 1e3
    df = df.sort_values("Time", ascending=True).reset_index(drop=True)
    df["Time"] = pd.to_datetime(df["Time"], unit="s")
    for c in df.columns:
        if c != "Time":
            df[c] = df[c].astype(float)
    return df

In [ ]:
def extract_cream_borrow_apy_history_from_file(filename: str, comptroller: str, symbols: str) -> pd.DataFrame():
    """
    Example:
    df_borrows = extract_cream_borrow_apy_history_from_file(
        "cream_finance_history_20220511.json",
        "avalanche",
        ("USDC.e", "WAVAX")
    )
    Or alternately replace first argument with the data object read in.
    """

    if not isinstance(filename, str):
        # Data passed in, not file
        data = filename
    else:
        with open(filename, "r") as f:
            data = json.load(f)
    
    dfs = []
    for sym in symbols:
        for dd in data:
            if dd[0]["underlying_symbol"] == sym and dd[0]["comptroller"].lower() == chain:
                break
        df_temp = pd.DataFrame(dd)[["date", "borrow_apy"]]
        df_temp["borrow_apy"] = df_temp["borrow_apy"].astype(float) / 100.0  # Cast to float and map to unit 1.0
        df_temp["date"] = pd.to_datetime(df_temp["date"], utc=True)
        df_temp = df_temp.rename({"borrow_apy": f"apy_borrow_{sym}"}, axis=1)
        dfs.append(df_temp)

    # Combine the two
    df_borrows = pd.merge(dfs[0], dfs[1], on="date", how="inner")
    df_borrows = df_borrows.sort_values("date", ascending=True).set_index("date", drop=True)
    
    return df_borrows

In [ ]:
def extract_coindix_apy_history_from_file(filename: str, chain: str, protocol: str, pair: str) -> pd.DataFrame():
    """
    Example: 
    df_coindix = extract_coindix_apy_history_from_file(
        "coindix_vault_history_20220511.json", 
        "avalanche",         
        "trader joe", 
        "USDC.e-AVAX"
    )
    """
    # Load and configure top-level columns
    df = pd.read_json(filename)
    df["chain"] = df["chain"].str.lower()
    df["protocol"] = df["protocol"].str.lower()

    # Filter to pair/protocol of interest
    dft = df[(df["name"] == pair) & (df["protocol"] == protocol) & (df["chain"] == chain)].iloc[0]["series"]
    dft = pd.DataFrame(dft)

    # Transform the columns
    for c in dft.columns:
        if c == "date":
            dft[c] = pd.to_datetime(dft[c], utc=True)
        else:
            dft[c] = dft[c].astype(float)  # Cast to float. Comes in mapped such that 0.1 is 10%
            
    return dft

<a id='scenario_generators'></a>
## Scenario Generators

In [ ]:
def create_no_price_change_example(n_days: int, token0_price: float = 1.0, token1_price: float = 0.1) -> pd.DataFrame:
    """
    Creates a n-day table where the price remains constant. Useful for testing interest without IL in the mix
    """
    df = initialize_price_dataframe(n_days, token0_price, token1_price)
    return df

In [ ]:
def create_linear_price_change_example(
    n_days: int, 
    token0_price: float = 1.0, 
    token1_price_initial: float = 0.1, 
    token1_price_final: float = 0.01,
    reward_token_price_initial: Optional[float] = None, 
    reward_token_price_final: Optional[float] = None,    
) -> pd.DataFrame:
    """
    Creates a n-day table where the both the non-safe-asset (NSA) and the reward token 
    price (optionally) linearly changes over time.
    """
    df = initialize_price_dataframe(n_days, token0_price, token1_price_initial)
    
    # Apply the price change
    df["token1_price"] = np.linspace(token1_price_initial, token1_price_final, len(df))

    # Apply the price change
    if reward_token_price_initial and reward_token_price_final:
        df["reward_token_price"] = np.linspace(reward_token_price_initial, reward_token_price_final, len(df))    
    
    return df

In [ ]:
def create_linear_and_back_example(
    n_days: int, 
    token0_price: float = 1.0, 
    token1_price_base: float = 0.1, 
    token1_price_peak: float = 0.01,
    reward_token_price_base: Optional[float] = None, 
    reward_token_price_peak: Optional[float] = None,    
) -> pd.DataFrame:
    """
    Creates a n-day table where the both the non-safe-asset (NSA) and the reward token 
    price (optionally) linearly changes over time. It goes from base -> peak -> return to base. 
    """
    df = initialize_price_dataframe(n_days, token0_price, token1_price_base)
    
    # Apply the price change
    n = int(round(len(df)/2)) + 1
    df["token1_price"] = np.hstack((np.linspace(token1_price_base, token1_price_peak, n), np.linspace(token1_price_peak, token1_price_base, n+1)))[:len(df)]

    # Apply the price change
    if reward_token_price_base and reward_token_price_peak:
        df["reward_token_price"] = np.hstack((np.linspace(reward_token_price_base, reward_token_price_peak, n), np.linspace(reward_token_price_peak, reward_token_price_base, n+1)))[:n]
    
    return df

In [ ]:
def create_random_walk_example(
    n_days: int,
    token1_price_initial: float, 
    bias: float, 
    variance: float, 
    reward_token_price_initial: Optional[float] = None,
    seed: Optional[int] = None
) -> pd.DataFrame:
    """
    Creates a random walk for token1 and optionally reward_token with a 
    defined origin, bias, and variance. token0 remains "stable" at $1.0
    Note: bias and variance are per step. Since it is a random walk, each run gives different outputs.
    """
    df = initialize_price_dataframe(n_days, 1.0, token1_price_initial)
    
    # Apply the price change
    df["token1_price"] = random_walk(n_days, token1_price_initial, bias, variance, seed=seed)

    # Apply the price change
    if reward_token_price_initial:
        seed2 = None
        if seed:
            seed2 = int(seed * 2)  # Arbitrary deterministic, but separate, seed for rewards
        df["reward_token_price"] = random_walk(n_days, reward_token_price_initial, bias, variance, seed=seed2)
    
    return df    
    

In [ ]:
def create_small_example() -> pd.DataFrame:
    """
    Creates a simple input where there are 4 records. 
    Records:
        1: Opening
        2: Identical to record 1, to confirm interest/appreciation without price change
        3: Asset price (token1) drops to 10% of initial value. To test calculations under IL.
        4: Asset price (token1) increases to 5X of initial value. To test calculations under IL in other direction
    """
    df = create_no_price_change_example(4, 1.0, 0.1)
    df.iloc[2,1] = 0.01
    df.iloc[3,1] = 0.5
    return df

In [ ]:
def create_coingecko_price_history(
    coin_id: str, 
    start_time: Optional[int] = None, 
    end_time: Optional[int] = None
) -> pd.DataFrame:
    """
    Fetches a symbol's price history from CoinGecko API. 
    See get_coingecko_price_history for example.
    Converts to this format.
    Note: Currently assumes a stable is the base token.
    """
    if start_time is None:
        start_time = 0    
    if end_time is None:
        end_time = int(time.time())
    s = get_coingecko_price_history(coin_id)
    
    # Filter by times
    start_ts = pd.Timestamp(start_time, unit="s")
    end_ts = pd.Timestamp(end_time, unit="s")
    s = s[start_ts:end_ts]
    
    # Pull out close events
    df = pd.DataFrame(s)
    df = df.rename({"price": "token1_price"}, axis=1)
    df["token0_price"] = 1.0
    df = df.reset_index()
    df = df.sort_values("date", ascending=True)
    df = df[["date", "token0_price", "token1_price"]]
    return df

In [ ]:
def create_binance_price_history(
    symbol: str, 
    candle_size: str = "1d", 
    start_time: Optional[int] = None, 
    end_time: Optional[int] = None
) -> pd.DataFrame:
    """
    Fetches a pair's history from Binance candlestick API. 
    See get_binance_klines for example.
    Converts to this format.
    Note: Currently assumes a stable is the base token.
    """
    if start_time is None:
        start_time = 0    
    if end_time is None:
        end_time = int(time.time())
    df_c = get_binance_klines(
        symbol, candle_size, start_time, end_time, 10000
    )
    
    # Pull out close events
    df = df_c[["Time", "Close"]]
    df = df.rename({"Time": "date", "Close": "token1_price"}, axis=1)
    df["token0_price"] = 1.0
    df = df[["date", "token0_price", "token1_price"]]
    return df

In [ ]:
def create_history_from_files(
    coindix_pair: str,
    coindix_protocol: str,
    chain: str,
    token0_cream_name: str,
    token1_cream_name: str,
    token1_coingecko_id: str,
    reward_token_coingecko_id: Optional[str] = None,
    coindix_file: str = DEFAULT_COINDIX_HISTORY_FILE,
    cream_file: str = DEFAULT_CREAM_HISTORY_FILE,
    reward_apy_ratio: Optional[float] = None):
    """
    This is a scenario generator that extracts real pool APY data, borrow APY data,
    and prices from various sources. It then cleans them up and merges them by date 
    in a format that is compatible with the simulator.
    
    * Pool APY data is derived a CoinDix history file, saved using this function:
        https://github.com/ClaudeF4491/crypto_data_fetchers/blob/b03f6655e8fc4baaba6b49c6ae38a700bc701cb2/adapters/apis/coindix.py#L197
    * Borrow APY data is derived from a Cream Finance history file, saved using this script:
        https://github.com/ClaudeF4491/crypto_data_fetchers/blob/main/scripts/download_history_cream.py
    * Price Data is derived from an API call to CoinGecko, via the get_coingecko_price_history() function in this notebook.
    
    Example Usage:
    
    df = create_history_from_files(
        "USDC.e-AVAX",
        "trader joe",
        "avalanche",
        "USDC.e", 
        "WAVAX",
        "avalanche",
        "joe",
        coindix_file="coindix_vault_history_20220601.json",
        cream_file="cream_finance_history_20220601.json"
    )
    
    """
    """
    Fetch CoinDix Pool APY History
    """
    
    df_coindix = extract_coindix_apy_history_from_file(
        coindix_file, 
        chain,
        coindix_protocol,
        coindix_pair
    )
    apy_reward = df_coindix["reward"]
    apy_trading_fee = df_coindix["apy"]
    if pd.isna(df_coindix["reward"]).all() or (df_coindix["reward"].max() <= 0) and reward_apy_ratio is not None:
        print(f"No rewards found for this CoinDix pair. Setting reward based on reward_apy_ratio={reward_apy_ratio}")
        # No rewards found. Assume one based on reward_apy_ratio
        apy_reward = df_coindix["apy"] * reward_apy_ratio
        apy_trading_fee = df_coindix["apy"] * (1.0 - reward_apy_ratio)
    df_coindix["apy_trading_fee"] = apy_trading_fee
    df_coindix["apy_reward"] = apy_reward
    df_coindix = df_coindix[["date", "apy_trading_fee", "apy_reward"]]

    # Fill nans
    df_coindix = df_coindix.fillna(0.0)
    
    # Set index
    df_coindix = df_coindix.set_index("date", drop=True)
    
    """
    Fetch Borrow APYs from CREAM History File
    """
    df_borrows = extract_cream_borrow_apy_history_from_file(
        cream_file,
        chain,
        (token0_cream_name, token1_cream_name)
    )

    # Map naming
    rep = dict()
    for i, c in enumerate(df_borrows.columns): 
        rep[c] = f"apy_borrow_token{i}"
    df_borrows = df_borrows.rename(rep, axis=1)
    
    """ 
    Merge the two, keeping only dates that are in both
    """
    df = df_coindix.join(df_borrows, how="inner")
    
    """
    Fetch token1 price data from CoinGecko
    """
    df_price = create_coingecko_price_history(
        token1_coingecko_id, 
        start_time=int(df.index[0].timestamp()), 
        end_time=int(df.index[-1].timestamp()+60*60*24)  # add one day for good measure
    )
    df_price["date"] = pd.to_datetime(df_price["date"], utc=True)
    df_price = df_price.set_index("date", drop=True)
    
    """
    Merge with token1 from coingecko
    """
    df = df.join(df_price, how="inner")
    
    """
    Optionally fetch reward token prices and merge
    """
    if reward_token_coingecko_id:
        df_reward = create_coingecko_price_history(
            reward_token_coingecko_id, 
            start_time=int(df.index[0].timestamp()), 
            end_time=int(df.index[-1].timestamp()+60*60*24)  # add one day for good measure
        )
        df_reward["date"] = pd.to_datetime(df_reward["date"], utc=True)
        df_reward = df_reward.set_index("date", drop=True)
        df_reward = df_reward.rename({"token1_price": "reward_token_price"}, axis=1)
        df_reward = df_reward[["reward_token_price"]]
        
        # Merge
        df = df.join(df_reward, how="inner")
    
    return df

<a id='reward_token_strategies'></a>
## Reward Token Strategies

In [ ]:
def sell_rewards(
    reward_tokens: float, 
    reward_price: float, 
    pool_tokens: Optional[Tuple[float, float]] = None,
    pool_prices: Optional[Tuple[float, float]] = None,
    amount: float = 1.0,
    fee_swap: float = 0.003,
    fee_gas: float = 0.0
) -> Tuple[float, Tuple[float, float], float, float]:
    if amount <= 0:
        return reward_tokens, pool_tokens, 0.0, 0.0
    
    # Derive rewards being sold
    reward_tokens_sell = amount * reward_tokens

    # Update rewards: Remove rewards being sold
    reward_tokens = reward_tokens - reward_tokens_sell    
    
    # Derive tokens remaining after swap fee taken from transaction
    reward_tokens_sell_after_fees = reward_tokens_sell * (1.0 - fee_swap)
    
    # Sell tokens (after fee) to cash
    cash = reward_tokens_sell_after_fees * reward_price

    # Derive gas fees, accumulated separately since assumed covered by an independent balance of protocol tokens
    fees = fee_gas * 1.0  # one transaction
    
    return reward_tokens, pool_tokens, cash, fees


In [ ]:
def compound_rewards(
    reward_tokens: float, 
    reward_price: float, 
    pool_tokens: Optional[Tuple[float, float]] = None,
    pool_prices: Optional[Tuple[float, float]] = None,
    amount: float = 1.0,
    fee_swap: float = 0.003,
    fee_gas: float = 0.0
) -> Tuple[float, Tuple[float, float], float, float]:   
    if amount <= 0:
        return reward_tokens, pool_tokens, 0.0, 0.0
    
    # Sell assigned amount of rewards to cash
    reward_tokens, _, cash, fees = sell_rewards(reward_tokens, reward_price, None, None, amount, fee_swap, fee_gas)
    
    # Split cash between the two tokens, buy, and increment
    for i in [0, 1]:
        cash_swap_after_fees = 0.5*cash * (1.0 - fee_swap)
        bought_tokens = cash_swap_after_fees / pool_prices[i]
        pool_tokens[i] += bought_tokens
    cash = 0
    
    # Derive gas fees, accumulated separately since assumed covered by an independent balance of protocol tokens
    fees += fee_gas * 2.0  # two additional transactions (one for each token swap for LP pair)
    fees += fee_gas * 1.0  # one additional transactions (one for entry or stake into LP)
    
    return reward_tokens, pool_tokens, cash, fees


## Base Actions

Base events such as add liquidity, remove liquidity, etc.

In [ ]:
def add_liquidity(
    capital: float, 
    leverage: float,
    pool_prices: Optional[Tuple[float, float]],
    fee_swap: float = 0.003,
    fee_gas: float = 0.0
) -> Tuple[Tuple[float, float], Tuple[float, float], float]:
    """
    Opens or adds to a LP position given a predefined amount.
    Returns updated pool supply, debt, cash remaining, and fees accumulated
    All accumulative bookkeeping should happen OUTSIDE this function.
    """
    token0_price, token1_price = pool_prices
    token_supply = [0, 0]
    token_debt = [0, 0]
    fees = 0
    
    """
    For simplicity, we'll assume only ONE swap fee is accrued to open the position for all leverage levels
    We'll take this off the top. Even if multiple positions are opened in actuality, the fee gets reduced
    proportionally as the capital is split.
    """
    capital = capital * (1.0 - fee_swap)
    
    # Open position differently based on leverage type
    if leverage == 1:
        # No leverage. Split capital in half and open a standard LP position
        # A swap is assumed here for both tokens to get balanced tokens for LP
        token0_supply = (capital/2) / token0_price
        token1_supply = (capital/2) / token1_price
        
        # Change to final format. Note: 1X is no debt
        token_supply = [token0_supply, token1_supply]
        fees = fee_gas  # Two swap gas to get the tokens + LP entry gas
        
    elif leverage == 2:
        """
        2X leverage is a single-position. 
        Borrow 100% of principal in token1, and nothing in token0.
        """
        # Borrow by adding to debt and increasing supply by same amount
        token0_debt = 0  # No debt on base token, just NSA
        token1_debt = capital / token1_price
        token1_supply = token1_debt

        # Initial capital is used to purchase the token0-side of the pool
        token0_supply = capital / token0_price
        
        # Change to final format. Note: 1X is no debt
        token_supply = [token0_supply, token1_supply]
        token_debt = [token0_debt, token1_debt]
        fees = 4 * fee_gas  # Two swap gas to get the tokens + One borrow + One LP entry gas
        
        
    elif leverage > 2:
        """
        More than 2X leverage requires two positions, or possibly a single position
        with dual-borrowing if allowed. (Ref: "Revisiting the Fundamentals of Pseudo-Delta Neutral Hedging")
        This aggregates tally into one single LP for bookkeeping purposes
        Ref: "Alpaca Finance Yield Farming Calculator", Sheet: "1.1) Pseudo delta-neutral LP farming"

        Assumes we start with principal in token0
        Example: 3X leverage @ $100, borrow ratio is L/(L-2):1, or 3:1 for 3X
        Position, total $300:
            tokenA = 0.5*principal*leverage = $150
            tokenB = 0.5*principal*leverage = $150
        Debt, total $200:
            tokenA = 0.25*principal*(leverage-1) = $50
            tokenB = 0.75*principal*(leverage-1) = $150
        """

        # Total supply ($) is leverage * initial capital. Split across tokens in LP pair
        token0_supply_value = 0.5 * leverage * capital
        token1_supply_value = 0.5 * leverage * capital

        # Derive number of tokens from those supplies
        token0_supply = token0_supply_value / token0_price
        token1_supply = token1_supply_value / token1_price

        # To open the positions that provide the supply above, we need to borrow (leverage-1) dollars
        multiplier = (leverage - 1)

        # The ratio in which we borrow depends on the leverage amount.
        # Specifically it is a ratio of  L/(L-2):1, where the left side is the risky asset
        # e.g. for 3X, it is 3:1, or 3/4 borrow risky, 1/4 borrow stable  
        token0_frac = 1 / (leverage / (leverage - 2) + 1)
        token1_frac = 1.0 - token0_frac

        # Derive the debt in dollars based on above
        token0_debt_value = capital * token0_frac * multiplier
        token1_debt_value = capital * token1_frac * multiplier

        # Derive number of tokens from those debts
        token0_debt = token0_debt_value / token0_price
        token1_debt = token1_debt_value / token1_price    

        # Change to final format. Note: 1X is no debt
        token_supply = [token0_supply, token1_supply]
        token_debt = [token0_debt, token1_debt]
        
        # Fees include: one borrow for each token (2), one swap for each borrow (2), two LP pool entries (2)
        # 6 events total
        fees = 6 * fee_gas  

    else:
        raise ValueError("Only supports leverage=1 or leverage>=2.")
    
    return token_supply, token_debt, fees

In [ ]:
def remove_liquidity(
    token_supply: Tuple[float, float], 
    token_debt: Tuple[float, float],
    pool_prices: Optional[Tuple[float, float]],    
    amount: float = 1.0, 
    fee_swap: float = 0.003,
    fee_gas: float = 0.0
) -> Tuple[Tuple[float, float], Tuple[float, float], float, float]:
    """
    Fully or partially closes LP position
    Returns remaining pool supply, remaining debt, cash returned, and fees accumulated
    All accumulative bookkeeping should happen OUTSIDE this function.
    Assumes two gas events (swapping out each token).
    
    amount is the ratio (0-1) of total equity to sell off.
    
    Note: This can be vectorized or placed in loop, but left as manual/explicit to 
    make the bookkeeping transparent
        
    """
    token0_price, token1_price = pool_prices
    token0_supply, token1_supply = token_supply
    token0_debt, token1_debt = token_debt
    fees = 0
    cash = 0
    
    # Determine what will remain after we remove the liquidity. Hold that aside.
    token0_supply_remaining = token0_supply * (1.0 - amount)
    token1_supply_remaining = token1_supply * (1.0 - amount)
    token0_debt_remaining = token0_debt * (1.0 - amount)
    token1_debt_remaining = token1_debt * (1.0 - amount)
    token_supply_remaining = [token0_supply_remaining, token1_supply_remaining]
    token_debt_remaining = [token0_debt_remaining, token1_debt_remaining]
    
    # Derive remaining tokens as well as cash accumulated based on equity
    # Token0 sell
    token0_equity = token0_supply  - token0_debt
    token0_sell = token0_equity * amount    
    token0_cash = token0_sell * token0_price * (1.0 - fee_swap)  # Include swap fee
    fees += fee_gas  # swap out token0

    # Token1 sell
    token1_equity = token1_supply - token1_debt
    token1_sell = token1_equity * amount
    token1_cash = token1_sell * token1_price  * (1.0 - fee_swap)  # Include swap fee
    fees += fee_gas  # swap out token1
    
    # Determine final cash from position removal
    cash = token0_cash + token1_cash
    
    return token_supply_remaining, token_debt_remaining, cash, fees
    

<a id='trade_strategies'></a>
## Trade Strategies

Strategies to execute on every epoch to swap, open, close, etc, based on pre-defined rules or conditions.

In [ ]:
class HODLStrategy():
    """ 
    Pass through strategy. HODL. No change over time. 
    """
    def __init__(self) -> None:
        pass
        
    def execute(
        self,
        token_supply: Tuple[float, float], 
        token_debt: Tuple[float, float],
        df: pd.DataFrame    
    ) -> Tuple[Tuple[float, float], Tuple[float, float], float, float]:
        return token_supply, token_debt, 0, 0


In [ ]:
class RebalanceStrategy():
    """ 
    Simple rebalancing strategy that rebalances when the current price moves up or down X% from the anchor.
    For simplicity, the rebalance isn't doing fancy math to add or remove collateral. It's just closing
    the position and re-opening. Normally fees are higher by doing that, but they can be dampened by adjusting
    fee_* during initialization.
    """
    def __init__(
        self, 
        price_anchor: float, 
        threshold: float,
        leverage: float,
        amount: float = 1.0,
        fee_swap: float = 0.003,
        fee_gas: float = 0.0
    ) -> None:
        self._price_anchor = price_anchor
        self._threshold = threshold
        self._leverage = leverage
        self._amount = amount
        self._fee_swap = fee_swap
        self._fee_gas = fee_gas
        self._initial_price = price_anchor
        
    def execute(
        self,
        token_supply: Tuple[float, float], 
        token_debt: Tuple[float, float],
        df: pd.DataFrame    
    ) -> Tuple[Tuple[float, float], Tuple[float, float], float, float]:
        """ 
        Execute strategy. 
        df is the main dataframe up to current date
        """
        
        # Get most recent price from data table
        token0_price = df.iloc[-1]["token0_price"]
        token1_price = df.iloc[-1]["token1_price"]
        pool_prices = [token0_price, token1_price]
        
        # Init
        cash = 0  # No change in cash since we re-open with the cash aquired from closure        
        fees = 0
        
        # Check deviation
        price_delta = np.abs(token1_price - self._price_anchor) / self._price_anchor
        
        # Rebalance if deviation exceeds threshold
        if price_delta > self._threshold:
            print(f"Price Delta {price_delta} exceeded threshold {self._threshold}. cur_price={token1_price}, anchor_price={self._price_anchor}. Rebalancing.")
            # print(f"Before Close: {token_supply}, {token_debt}, {pool_prices}")
            # Close the position
            token_supply, token_debt, cash_close, fees_close = remove_liquidity(
                token_supply, 
                token_debt,
                pool_prices,    
                self._amount, 
                self._fee_swap,
                self._fee_gas
            )
            # print(f"After Close: {token_supply}, {token_debt}, {cash_close}, {fees_close}")
            
            # Re-open the position
            token_supply, token_debt, fees_open = add_liquidity(
                cash_close, 
                leverage,
                pool_prices,
                self._fee_swap,
                self._fee_gas
            )
            # print(f"After Open: {token_supply}, {token_debt}, {fees_open}")
            
            # Accumulate cash and fees
            fees = fees_close + fees_open

            # Update the anchor price to be current
            self._price_anchor = token1_price
        
        return token_supply, token_debt, cash, fees

<a id='configuration_setup'></a>
# Configuration & Scenario Setup
---

## Parameters

In [ ]:
"""
Configuration that is always used, regardless of scenario
"""
# Initial amount invested
initial_cap = 10000

# Swap fee, note: 0.01 = 1%
fee_swap = 0.00

# Cost of gas for each transaction, in dollars
fee_gas = 0

# Open a leveraged position
# Options: [1.0, 2.0, >2.0]
leverage = 3

# Price of reward token
reward_token_price = 1.0

# Liquidation threshold
# Note: Francium threshold is 83.3%. See Francium reference at top.
# Currently unused. For plotting purposes only.
liquidation_threshold = 0.833

"""
Configuration that is used if the scenario does not internally generate the data itself
"""
# Earnings APYs, note: 1.0 = 100%
apy_trading_fee = 0.2
apy_reward = 0.2

# Borrow / Lending APYs
apy_borrow_token0 = 0.1
apy_borrow_token1 = 0.1


"""
Configuration that is used if `create_history_from_files()` is selected as the scenario. 
That generator derives all pool APYs, borrow APYs, and prices
"""
# Token names, according to CREAM, to lookup borrow rates. 
# May differ from CoinDix pool symbol name
cream_token0_name = "USDC.e" 
cream_token1_name = "WAVAX"

# CoinGecko config
token1_coingecko_id = "avalanche-2"
reward_token_coingecko_id= "joe"

# CoinDix parameters
coindix_pair = "USDC.e-AVAX"
protocol = "trader joe"
chain = "avalanche"

<a id='reward_strategy_definition'></a>
## Reward Strategy Definition
Define a reward function using one found in [Reward Token Strategies](#reward_token_strategies) section. Or create your own and reference it here.

In [ ]:
# Define parameters for what to do with rewards
# To HODL, use any function with rewards_sell_amount=0
rewards_fn = compound_rewards  # Options: compound_rewards, sell_rewards
rewards_sell_amount = 1.0

<a id='price_movement_scenario'></a>
## Price Movement Scenario

Uncomment the scenario to try it. Or create your own from a function in the [Scenario Generators](#scenario_generators) section.

In [ ]:
"""
Uncomment to create a small 4-day example that unit-tests: no change, very negative price change, very positive price change
"""
# pp = create_small_example()


"""
Uncomment this to run for one year without any price change -- useful to test APY growths and fees without price impact
"""
# pp = create_no_price_change_example(365)


"""
Uncomment this to run for one year, decaying price change to 80% losses -- useful to see dramatic effects on investment
"""
# pp = create_linear_price_change_example(365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.02)


"""
Uncomment this to run for one year, increasing price 5X -- useful to see dramatic effects on investment
"""
# pp = create_linear_price_change_example(365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.5)


"""
Uncomment this to run for one year, without any non-safe-asset (NSA) price change, but the reward token drains to near zero.
This is an example of typical farming token where it may be better to sell every day instead of compound or hold, 
depending on initial capital and gas fees.
"""
# pp = create_linear_price_change_example(
#     365, token0_price=1.0, token1_price_initial=0.1, token1_price_final=0.1, reward_token_price_initial=1.0, reward_token_price_final=0.01
# )

"""
Uncomment this to go from price of 1.0 down to 0.2 and back to 0.1 within a year. Tests to see the effect of returning to no IL 
when price dumps and comes back
"""
# pp = create_linear_and_back_example(365, token0_price=1.0, token1_price_base=0.1, token1_price_peak=0.01)

"""
Uncomment this to go from price of 1.0 down to 5.0 (5X) and back to 0.1 within a year. Tests to see the effect of returning to 
no IL when price pumps and comes back
"""
pp = create_linear_and_back_example(365, token0_price=1.0, token1_price_base=0.1, token1_price_peak=0.3)

"""
Uncomment this to take a random walk. 
Tips: 
- Set seed to None for random every time, or integer to be deterministic
- Set bias and variance based in terms of daily-percent movement. 
- See defaults for reasonable starts. Try plotting first as well.
- Set bias to positive to walk upward, negative to walk downward, or 0 to randomly move stationary. 
- Set variance to define how much the walk should move each step.
"""
# pp = create_random_walk_example(365, token1_price_initial=10, bias=-0.002, variance=0.001, seed=1234)

"""
Uncomment this to run a real-world example using actual historical data.
Reads in AVAX-USDC.e pool example from CoinDix/CREAM and prices from coingecko
Assumes 10% of total APY is rewards (coindix doesn't report reward APY for this pool, so just an assumption)
"""
# pp = create_history_from_files(
#     coindix_pair,
#     protocol,
#     chain,
#     cream_token0_name, 
#     cream_token1_name,
#     token1_coingecko_id,
#     reward_token_coingecko_id,
#     reward_apy_ratio=0.10
# )

# Suppress auto-comment print above
print("")

<a id='trade_strategy_definition'></a>
## Trade Strategy Definition
Initialize a trade strategy object found in [Trade Strategies](#trade_strategies)

In [ ]:
"""
This strategy is a no-op. It takes no action and is just a HODL.
"""
# strategy_cls = HODLStrategy()

"""
This strategy rebalances whenever the price moves 5% from the last rebalance price. It does this by just closing and re-opening
the position. In reality, this would be done through collateral rebalancing but the net effect is the same. To accommodate for 
this difference, gas and swap fees are reduced here.
"""
strategy_cls = RebalanceStrategy(
        pp["token1_price"][0], 
        50.4,
        leverage,
        1.0,
        fee_swap=fee_swap/10.0,  # In real-world, the swap will be a balancing of a small percentage of the total position
        fee_gas=fee_gas/3.0
)

# Suppress odd comment prints
print("")

# Initialize
---

## Initialize Input DataFrame

In [ ]:
# Add derived columns to dataframe
if "reward_token_price" not in pp.columns:
    pp["reward_token_price"] = reward_token_price
if "apy_trading_fee" not in pp.columns:
    pp["apy_trading_fee"] = apy_trading_fee
if "apy_reward" not in pp.columns:    
    pp["apy_reward"] = apy_reward
if "apy_borrow_token0" not in pp.columns:    
    pp["apy_borrow_token0"] = apy_borrow_token0
if "apy_borrow_token1" not in pp.columns:        
    pp["apy_borrow_token1"] = apy_borrow_token1
pp["fee_gas"] = fee_gas

## Initialize position and pool settings

In [ ]:
price0_initial = pp["token0_price"][0]
price1_initial = pp["token1_price"][0]
ratio_initial = price1_initial / price0_initial

# Open the position
token_supply, token_debt, fees_open = add_liquidity(initial_cap, leverage, [price0_initial, price1_initial], fee_swap, fee_gas)
token0_supply_initial, token1_supply_initial = token_supply
token0_debt_initial, token1_debt_initial = token_debt

# Calculate the product constant of the liquidity pool
x = token0_supply_initial
y = token1_supply_initial
k = x * y

## Initialize Columns

In [ ]:
# Initialize the columns we will fill in iteratively
cols = [
    "token0_supply_open",
    "token1_supply_open",
    "position_pool_open_dollars",
    "apy_total",
    "token0_earnings",
    "token1_earnings",
    "trading_fee_earnings_dollars",
    "reward_earnings_dollars",
    "reward_token_earnings",
    "cash_from_rewards",
    "token0_supply_close",
    "token1_supply_close",
    "token0_debt_close",
    "token1_debt_close",    
    "pool_value",
    "cash_value", 
    "rewards_value", 
    "fees_value", 
    "position_value",
    "debt_value",
    "equity_value",
    "effective_leverage",
    "pool_equity",
    "profit_value",
    "roi",
    "annualized_apr",
    "debt_ratio",
    "iloss",
    "position_hodl_dollars",
    "trade_event",
]
pp[cols] = np.nan
pp["trade_event"] = False

# Run Simulation
---

In [ ]:
"""
Calculate pool supply based on change in prices

pre: Before daily rewards accounted for, but after impermanent loss token shift accounted for
post: After daily rewards accounted for and after impermanent loss

Ref: https://docs.google.com/spreadsheets/d/15pHFfo_Pe66VD59bTP2wsSAgK-DNE_Xic8HEIUY32uQ/edit#gid=650365.25877
    (Alpaca Finance Yield Farming Calculator)
"""
# Initialize before starting loop. Recall position was already opened above.
fees_accum = fees_open
cash_accum = 0
rewards_accum = 0
last_token0_supply = token0_supply_initial
last_token1_supply = token1_supply_initial
last_token0_debt = token0_debt_initial
last_token1_debt = token1_debt_initial
first_date = pp.iloc[0].name

# Loop epoch-by-epoch and calculate changes over time
for idx, row in pp.iterrows():
    # Derive ratio of token B divided by token A. Provides price of A relative to B
    cur_date = row.name
    days_elapsed = (cur_date - first_date).days + 1
    token0_price = row["token0_price"]
    token1_price = row["token1_price"]
    reward_token_price = row["reward_token_price"]
    ratio = row["token1_price"] / row["token0_price"]
    apy_trading_fee = row["apy_trading_fee"]
    apy_reward = row["apy_reward"]
    fee_gas = row["fee_gas"]
        
    # Calculate number of tokens in pool, after impermanent loss, each step. 
    tk = last_token0_supply * last_token1_supply  # Derive constant based on tokens in pool at close of last step
    token0_supply_open, token1_supply_open = iloss_amt(tk, ratio)  # Calculate tokens at open given impermanent loss due to new price
    
    # Define intermediate values that change during epoch based on strategies
    token0_supply_cur = token0_supply_open
    token1_supply_cur = token1_supply_open
    token0_debt_cur = last_token0_debt
    token1_debt_cur = last_token1_debt

    """
    Calculate positions (pre), before rewards
    """

    # Given current tokens in pool, calculate equity from the LP
    position_pool_open_dollars = token0_price * token0_supply_open + token1_price * token1_supply_open

    """
    Derive APYs
    """
    apy_total = apy_trading_fee + apy_reward


    """
    Calculate earnings
    """

    # Trading Fee Earnings
    # Accrue tokens in pool assuming using daily compounded trading fee APY (see reference at top)
    token0_earnings = token0_supply_open * (apy_trading_fee / 365.25)  # Token0 daily increase due to trading fees
    token1_earnings = token1_supply_open * (apy_trading_fee / 365.25)  # Token1 daily increase due to trading fees
    trading_fee_earnings_dollars = token0_earnings * token0_price + token1_earnings * token1_price
    
    # Accrue
    token0_supply_cur += token0_earnings
    token1_supply_cur += token1_earnings
    

    # Farming Rewards
    # Assign to tokens since that's what we are rewarded in   
    # Assumes rewards based on starting value of the pool, before trading fees for the day
    reward_earnings_dollars = position_pool_open_dollars * apy_reward/365.25
    reward_token_earnings = reward_earnings_dollars / reward_token_price
 
    
    """
    Calculate debt
    """
    # Debt is accrued daily in the form of number of tokens that are owed here
    # interest accrued = n_borrowed_tokens * daily_interest_rate
    # compounded continuously, ref: https://www.crunchbase.com/organization/ethlend
    token0_debt_cur = token0_debt_cur * np.exp(apy_borrow_token0 * 1.0/365.25)
    token1_debt_cur = token1_debt_cur * np.exp(apy_borrow_token1 * 1.0/365.25)
    
    
    """
    Apply reward strategy
    """
    
    pool_tokens_from_rewards = [0, 0]
    reward_tokens_remaining, pool_tokens_from_rewards, cash_rewards, fees_rewards = rewards_fn(
        reward_token_earnings,
        reward_token_price,
        pool_tokens_from_rewards,
        [token0_price, token1_price],
        rewards_sell_amount, 
        fee_swap=fee_swap,
        fee_gas=fee_gas
    )
    
    # Accrue
    token0_supply_cur += pool_tokens_from_rewards[0]
    token1_supply_cur += pool_tokens_from_rewards[1]       
    
    """
    Apply trading strategy
    """
    
    pp_input = pp[pp.index<=idx]
    token_supply_in = [token0_supply_cur, token1_supply_cur]
    token_debt_in = [token0_debt_cur, token1_debt_cur]
    token_supply_out, token_debt_out, cash_trade, fees_trade = strategy_cls.execute(token_supply_in, token_debt_in, pp_input)
    
    # Track the event if a trade occurred
    trade_occurred = False
    if (token_supply_in != token_supply_out) or (token_debt_in != token_debt_out):
        trade_occurred = True
        
    
    # Accrue
    token0_supply_cur, token1_supply_cur = token_supply_out
    token0_debt_cur, token1_debt_cur = token_debt_out
    
    """
    Accumulate the other books (fees, cash, rewards)
    """
    
    # Accumulate
    rewards_accum += reward_tokens_remaining
    cash_accum = cash_accum + cash_rewards + cash_trade    
    fees_accum = fees_accum + fees_rewards + fees_trade
    
    """
    Update token balances based on previous, trading-fee earnings, and any compounded rewards
    """

    # Close out using latest intermediate data
    token0_supply_close = token0_supply_cur
    token1_supply_close = token1_supply_cur
    token0_debt_close = token0_debt_cur
    token1_debt_close = token1_debt_cur
    
    # With owed-tokens incremented, calculate the dollar value for each
    token0_debt_value = token0_debt_close * token0_price
    token1_debt_value = token1_debt_close * token1_price   
        
    """
    Calculate positions (post), including daily rewards
    """
    # Derive final position total
    # position includes: pool value, rewards value, cash value
    pool_value = token0_price * token0_supply_close + token1_price * token1_supply_close
    cash_value = cash_accum
    fees_value = fees_accum
    rewards_value = rewards_accum * reward_token_price
    position_value = pool_value + rewards_value + cash_value - fees_value  # Position of LP + unclaimed rewards + cash - fees incurred
    debt_value = token0_debt_value + token1_debt_value  # Debt from LP position borrowing
    pool_equity = pool_value - debt_value
    equity_value = position_value - debt_value  # Equity in LP positions
    profit_value = equity_value - initial_cap  # PnL is total gains/losses minus initial investment
    roi = profit_value / initial_cap
    debt_ratio = debt_value / (debt_value + pool_equity)  # See francium reference. We do not include rewards or cash here since they aren't in the pool.
    annualized_apr = (1.0 + (profit_value / initial_cap)) ** (365 / days_elapsed) - 1
    effective_leverage = pool_value / pool_equity 
    
    """
    Calulate raw impermament loss, without considering rewards
    """

    # Derive ratio now vs. what it was at start
    rel_ratio = ratio_initial / ratio

    # Calculate impermanent loss directly from relative ratios
    iloss = 2 * (rel_ratio**0.5 / (1 + rel_ratio)) - 1


    """
    Calculate HODL position as a benchmark (independent of all else above)
    """

    # Calculate what we would have had if we just HODL'd the original tokens (x, y)
    position_hodl_dollars = (token0_price * token0_supply_initial + token1_price * token1_supply_initial) / leverage

    
    """
    Record current state
    """    
    
    # Create a record of all the derived values in this epoch
    daily_record = {
        "token0_supply_open": token0_supply_open,
        "token1_supply_open": token1_supply_open,
        "position_pool_open_dollars": position_pool_open_dollars,
        "apy_total": apy_total,
        "token0_earnings": token0_earnings,
        "token1_earnings": token1_earnings,
        "trading_fee_earnings_dollars": trading_fee_earnings_dollars,
        "reward_earnings_dollars": reward_earnings_dollars,
        "reward_token_earnings": reward_token_earnings,
        "cash_from_rewards": cash_rewards,
        "token0_supply_close": token0_supply_close,
        "token1_supply_close": token1_supply_close,
        "token0_debt_close": token0_debt_close,
        "token1_debt_close": token1_debt_close,
        "pool_value": pool_value,
        "cash_value": cash_value,
        "rewards_value": rewards_value,
        "fees_value": fees_value,
        "position_value": position_value,
        "debt_value": debt_value,
        "pool_equity": pool_equity,        
        "equity_value": equity_value,
        "profit_value": profit_value,
        "effective_leverage": effective_leverage,
        "roi": roi,
        "annualized_apr": annualized_apr, 
        "debt_ratio": debt_ratio,
        "iloss": iloss,
        "position_hodl_dollars": position_hodl_dollars,
        "trade_event": trade_occurred
    }
    
    # Update the epoch in the dataframe to include these derived values
    pp.loc[idx] = {**pp.loc[idx].to_dict(), **daily_record}

    # Save state for next step
    last_token0_supply = token0_supply_close
    last_token1_supply = token1_supply_close
    last_token0_debt = token0_debt_close
    last_token1_debt = token1_debt_close


# Table Summary
---

In [ ]:
pp.T

# Text Summary
---

In [ ]:
final_row = pp.iloc[-1]
print(f"Initial Wallet Value: ${initial_cap:0.2f}")
print(f"Days Elapsed: {days_elapsed}")
print(f"Number of Trade Strategy Executions: {pp['trade_event'].sum()}")
print(f"Final Wallet Value: ${final_row['equity_value']:0.2f}")
print(f"Final Profit: ${final_row['profit_value']:0.2f}")
print(f"Effective APR: {final_row['annualized_apr'] * 100:0.2f}%")

# Plot Results
---

In [ ]:
# Filter trade events and anchor to top of plot
tradey = pp["token1_price"].max()
trade_points_plot = pp["trade_event"] * tradey
trade_points_plot = trade_points_plot[trade_points_plot>0]

fig = go.Figure()
fig.add_trace(go.Scatter(x=pp.index, y=pp["token1_price"], mode='lines', name='token1_price'))
fig.add_trace(go.Scatter(
    x=trade_points_plot.index, y=trade_points_plot, mode='markers', name='Trade Events', marker_line_width=2, marker_size=7
))
fig.update_layout(
    title="Price of token1 over time, and when Trade Strategy events executed",
    xaxis_title="Price ($)",
    yaxis_title="date"
)


In [ ]:
px.line(pp, y="token1_price", title=f"Token 1 Price Over Time, final=${pp['token1_price'][-1]:.2f}")

In [ ]:
px.line(pp, y="reward_token_price", title=f"Reward Token Price Over Time, final=${pp['reward_token_price'][-1]:.2f}")

In [ ]:
print("Note: Total Equity = Pool Equity + Cash + Unrealized Rewards - Fees")
px.line(pp, y=["equity_value"], title=f"Total Equity Value Over Time, final=${pp['equity_value'][-1]:.2f}")

In [ ]:
px.line(pp, y=["token0_supply_close", "token1_supply_close", "token0_debt_close", "token1_debt_close",], title=f"Token Supply and Debt Over Time")

In [ ]:
px.line(pp, y=["debt_value", "position_value"], title="Position and Debt Value Over Time")

In [ ]:
fig = px.line(pp, y=["effective_leverage"], title=f"Effective Leverage Over Time (Pool Position / Pool Equity), final={pp['effective_leverage'][-1]:.2f}")
fig.update_yaxes(range=[0, 3.5], autorange=True)

In [ ]:
px.line(pp, y=["profit_value"], title=f"Profit Over Time, final=${pp['profit_value'][-1]:.2f}")

In [ ]:
fig = px.line(pp, y="roi", title=f"ROI % On a Given Date, final={pp['roi'][-1]*100:.2f}%")
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

In [ ]:
px.line(pp, y=pp["annualized_apr"]*100, title=f"Annualized APR At Point in Time, final={pp['annualized_apr'][-1]*100:.2f}%")

In [ ]:
fig = px.line(pp, y=["apy_trading_fee", "apy_reward"], title="Accrual APYs Over Time")
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

In [ ]:
fig = px.line(pp, y=["apy_borrow_token0", "apy_borrow_token1"], title=f"Borrow APYs Over Time")
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

In [ ]:
fig = px.line(pp, y="iloss", title=f"Impermanent Loss Over Time (price change only, unrelated to position value), final={pp['iloss'][-1]*100:.2f}%")
fig.layout.yaxis.tickformat = ',.0%'
fig.show()

In [ ]:
f"Rewards Value and Cash Value Accumulated, final rewards_value = ${pp['rewards_value'][-1]:.2f}, final cash_value = ${pp['cash_value'][-1]:.2f}"

In [ ]:
px.line(pp, y=["rewards_value", "cash_value"], title=f"Rewards Value and Cash Value Accumulated, finals = [${pp['rewards_value'][-1]:.2f}, {pp['cash_value'][-1]:.2f}]")

In [ ]:
px.line(pp, y=["fees_value"], title=f"Gas Fees Accumulated, final = ${pp['fees_value'][-1]:.2f}")

In [ ]:
print("Debt Ratio here is defined as `debt_value / (debt_value + pool_equity)`. Since it's used to consider liquidation, it is a function of the LP alone, and not fees/rewards/cash.")
fig = px.line(pp, y=["debt_ratio"], title=f"Debt Ratio over Time, final = {pp['debt_ratio'][-1]:3f}")
fig.add_hline(y=liquidation_threshold, line_width=3, line_dash="dash", line_color="red")
fig.update_yaxes(range=[0, 1], autorange=False)
fig.show()

In [ ]:
px.line(pp, y=["position_hodl_dollars"], title=f"Value if HODL'd original position (no LP), final = ${pp['position_hodl_dollars'][-1]:.2f}")